<a href="https://colab.research.google.com/github/hamednasr/TensorFlow-Projects/blob/main/Analyzing_Tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predicting Tweet`s Overal Message:
## This is a kaggle competition.
https://www.kaggle.com/competitions/nlp-getting-started/data

In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import pandas as pd
import random
from sklearn.model_selection import train_test_split

In [3]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2022-09-01 07:13:36--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-09-01 07:13:37 (117 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [42]:
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

In [5]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

--2022-09-01 07:13:37--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.16.128, 142.251.163.128, 142.250.188.48, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.16.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.004s  

2022-09-01 07:13:37 (161 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [6]:
unzip_data('/content/nlp_getting_started.zip')

## Data Exploration:

In [7]:
train_df = pd.read_csv('train.csv')
train_df

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [8]:
train_df['target'].value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [9]:
test_df = pd.read_csv('test.csv')
test_df

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


In [10]:
random_data = train_df.sample(frac=0.0006)
for i in range(len(random_data)):
  print(f'tweet number {i+1} :')
  if random_data.iloc[i, 4]==1:
    target = 'a Disaster'
  else:
    target = 'NOT Disaster'
  print(f'{random_data.iloc[i, 3]} : This is {target} tweet')
  print('------------------------------------\n')


tweet number 1 :
#SCSeEstaPreparando Light mag. 4.4 earthquake  - - 73km SW of Khuzdar Pakistan on Wednes... http://t.co/i6lmcccLv5 via @volcanodiscover : This is a Disaster tweet
------------------------------------

tweet number 2 :
95-03 BMW 528 530 540 740 Emergency Warning Hazard Switch Button OEM 20177-707D http://t.co/kVNahTHUWZ http://t.co/Y8xkNpqMnJ : This is NOT Disaster tweet
------------------------------------

tweet number 3 :
@kinkyconnors IM sorry for my meltdown last night lmao but I'm getting my tooth fixed Friday ?????????????????????????????????? : This is NOT Disaster tweet
------------------------------------

tweet number 4 :
You Bitches Walking Around Like Yall Hot Shit &amp; Got Bed Bugs &amp; You Burning ???? : This is NOT Disaster tweet
------------------------------------

tweet number 5 :
Excited not only about the next 6 years of school and ensuing student debt but also catastrophic climate change in my lifetime   

:D ?? : This is NOT Disaster tweet
-----

## Extract validation data from train data:

In [11]:
train_df['text'].to_numpy()

array(['Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all',
       'Forest fire near La Ronge Sask. Canada',
       "All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected",
       ...,
       'M1.94 [01:04 UTC]?5km S of Volcano Hawaii. http://t.co/zDtoyd8EbJ',
       'Police investigating after an e-bike collided with a car in Little Portugal. E-bike rider suffered serious non-life threatening injuries.',
       'The Latest: More Homes Razed by Northern California Wildfire - ABC News http://t.co/YmY4rSkQ3d'],
      dtype=object)

In [12]:
train_df['target'].to_numpy()

array([1, 1, 1, ..., 1, 1, 1])

In [13]:
train_data, val_data, train_label, val_label = train_test_split(train_df['text'].to_numpy(), train_df['target'].to_numpy(), 
                                                                test_size=0.1, 
                                                                random_state=42)

### extractin 10 percent of data from traid data and train lables:

In [14]:
random_num = np.random.choice(len(train_data),size=761,replace=False)

In [15]:
train_label_10_percent = train_label[random_num]

In [16]:
train_data_10_percent = train_data[random_num]

In [17]:
val_data[:10], val_label[:10]

(array(['So you have a new weapon that can cause un-imaginable destruction.',
        'The f$&amp;@ing things I do for #GISHWHES Just got soaked in a deluge going for pads and tampons. Thx @mishacollins @/@',
        'DT @georgegalloway: RT @Galloway4Mayor: \x89ÛÏThe CoL police can catch a pickpocket in Liverpool Stree... http://t.co/vXIn1gOq4Q',
        'Aftershock back to school kick off was great. I want to thank everyone for making it possible. What a great night.',
        'in response to trauma Children of Addicts develop a defensive self - one that decreases vulnerability. (3',
        '@Calum5SOS you look like you got caught in a rainstorm this is amazing and disgusting at the same time',
        'my favorite lady came to our volunteer meeting\nhopefully joining her youth collision and i am excite http://t.co/Ij0wQ490cS',
        '@brianroemmele UX fail of EMV - people want to insert and remove quickly like a gas pump stripe reader. 1 person told me it crashed the POS',
       

## Tokenization (Vectorization):

In [18]:
sum([len(tweet.split()) for tweet in train_data])

101905

In [19]:
round(sum([len(tweet.split()) for tweet in train_data])/len(train_data))

15

In [20]:
text_vectorizer = tf.keras.layers.TextVectorization(max_tokens=10000,
                                                       output_mode='int', 
                                                       output_sequence_length=15)

In [21]:
text_vectorizer.adapt(train_data)

In [22]:
sample_tweet = 'my name is hamed i am happy to meet you!'
text_vectorizer(sample_tweet)

<tf.Tensor: shape=(15,), dtype=int64, numpy=
array([  13,  707,    9,    1,    8,  171,  714,    5, 1815,   12,    0,
          0,    0,    0,    0])>

In [23]:
random_tweet = random.choice(train_data)
print('tweet: ', random_tweet,'\n\n')
print('vectorized tweet: ', text_vectorizer(random_tweet))

tweet:  @TinyJecht Are you another Stand-user? If you are I will have to detonate you with my Killer Queen. 


vectorized tweet:  tf.Tensor(
[3317   21   12  162 1977   46   12   21    8   42   24    5  440   12
   14], shape=(15,), dtype=int64)


In [24]:
#unique words
unique_words = text_vectorizer.get_vocabulary()
print(len(unique_words))
print(unique_words[:10])
print(unique_words[-10:])

10000
['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is']
['palm', 'palinfoen', 'palestinian\x89Û', 'palermo', 'pale', 'pakthey', 'pakistan\x89Ûªs', 'pakistans', 'pajamas', 'paints']


## Embedding:

In [25]:
embedding = tf.keras.layers.Embedding(input_dim = 10000, output_dim =32, input_length=None)

In [26]:
sample_embed = embedding(text_vectorizer(random_tweet))
sample_embed

<tf.Tensor: shape=(15, 32), dtype=float32, numpy=
array([[ 0.04075057,  0.00157804, -0.03359405,  0.04508943,  0.03111834,
        -0.01455407,  0.04644618,  0.01502276,  0.04720625, -0.04778763,
         0.04252073, -0.02199866,  0.00584967, -0.03747506,  0.02827128,
         0.01119568, -0.01374953, -0.02056975, -0.02447296,  0.02367014,
         0.032235  ,  0.04341171, -0.02728651, -0.00681353, -0.04186001,
         0.0005395 , -0.03170701, -0.00423217,  0.04575553,  0.03582663,
        -0.03258234, -0.02307471],
       [ 0.03013246,  0.03410004,  0.02283862,  0.01099455, -0.03165706,
        -0.01731058, -0.04910528,  0.00321706, -0.04119165, -0.01070045,
         0.01594395,  0.0049191 ,  0.0299284 , -0.01307793,  0.04405684,
         0.03071601,  0.03278383,  0.04952434,  0.03038255, -0.00617156,
         0.01923997,  0.02546281, -0.01654215,  0.00337489, -0.00709574,
        -0.02629893,  0.00700828, -0.04579277, -0.01228245,  0.03981679,
        -0.00177226,  0.04122628],
    

## Analyze with Naive Bayes using TF-IDF:

In [54]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report

In [55]:
model_0 = Pipeline([
                    ('tfidf', TfidfVectorizer()),
                    ('clf', MultinomialNB())
])

model_0.fit(train_data,train_label)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [56]:
val_pred = model_0.predict(val_data)
val_pred[:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1])

In [57]:
val_label[:10]

array([1, 0, 1, 0, 0, 0, 1, 1, 0, 1])

In [58]:
accuracy_score(val_label, val_pred)

0.7782152230971129

In [59]:
print(classification_report(val_label, val_pred))

              precision    recall  f1-score   support

           0       0.74      0.92      0.82       426
           1       0.86      0.60      0.70       336

    accuracy                           0.78       762
   macro avg       0.80      0.76      0.76       762
weighted avg       0.79      0.78      0.77       762



In [60]:
tweets=['earthquake hit california minutes ago', 
        'my car on fire now', 
        'I need a blue car']

In [61]:
model_0.predict_proba(tweets)

array([[0.22063312, 0.77936688],
       [0.58453661, 0.41546339],
       [0.71572258, 0.28427742]])

In [62]:
model_0.predict(tweets)

array([1, 0, 0])

## Analyze with Vanilla Neural Network:

In [84]:
# sequential API

# model_1 = tf.keras.Sequential([
#     tf.keras.layers.Input(shape= (1,), dtype = tf.string),
#     text_vectorizer,
#     embedding,
#     tf.keras.layers.GlobalAveragePooling1D(),
#     tf.keras.layers.Dense(10, activation='relu'),
#     tf.keras.layers.Dense(1, activation='sigmoid')    
# ])

# model_1.compile(loss = tf.keras.losses.BinaryCrossentropy(),
#                  optimizer = tf.keras.optimizers.Adam(),
#                  metrics= ['accuracy'])

In [165]:
# Functional API

inputs = tf.keras.layers.Input(shape= (1,), dtype = tf.string)

x = text_vectorizer(inputs)

x = embedding(x)

# x = tf.transpose(x , perm = [0,2,1])

# print(x.shape)

x = tf.keras.layers.GlobalAveragePooling1D()(x)

x = tf.keras.layers.Dense(10, activation='relu')(x)

outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model_1 = tf.keras.Model(inputs,outputs, name='ANN')

In [166]:
model_1.summary()

Model: "ANN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding_1 (Embedding)     (None, 15, 32)            320000    
                                                                 
 global_average_pooling1d_9   (None, 32)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_18 (Dense)            (None, 10)                330       
                                                                 
 dense_19 (Dense)            (None, 1)                 11      

In [167]:
model_1.compile(loss = tf.keras.losses.BinaryCrossentropy(),
                 optimizer = tf.keras.optimizers.Adam(),
                 metrics= ['accuracy'])

In [168]:
# !rm -rf './tensorboard/ANN'

In [171]:
model_1.fit(train_data_10_percent, 
            train_label_10_percent, 
            validation_data = (val_data,val_label), 
            epochs=10, 
            callbacks=[create_tensorboard_callback('tensorboard','ANN')])

Saving TensorBoard log files to: tensorboard/ANN/20220901-062035
Epoch 1/10
24/24 [==============================] - 0s 12ms/step - loss: 0.0439 - accuracy: 0.9803 - val_loss: 1.0258 - val_accuracy: 0.7375
Epoch 2/10
24/24 [==============================] - 0s 10ms/step - loss: 0.0345 - accuracy: 0.9908 - val_loss: 1.0437 - val_accuracy: 0.7323
Epoch 3/10
24/24 [==============================] - 0s 11ms/step - loss: 0.0290 - accuracy: 0.9908 - val_loss: 1.0485 - val_accuracy: 0.7310
Epoch 4/10
24/24 [==============================] - 0s 11ms/step - loss: 0.0259 - accuracy: 0.9934 - val_loss: 1.0509 - val_accuracy: 0.7336
Epoch 5/10
24/24 [==============================] - 0s 10ms/step - loss: 0.0234 - accuracy: 0.9934 - val_loss: 1.0535 - val_accuracy: 0.7349
Epoch 6/10
24/24 [==============================] - 0s 12ms/step - loss: 0.0215 - accuracy: 0.9947 - val_loss: 1.0657 - val_accuracy: 0.7349
Epoch 7/10
24/24 [==============================] - 0s 12ms/step - loss: 0.0202 - accurac

In [170]:
history_1 = model_1.fit(train_data, 
            train_label, 
            validation_data = (val_data,val_label), 
            epochs=10, 
            callbacks=[create_tensorboard_callback('tensorboard','ANN')])

Saving TensorBoard log files to: tensorboard/ANN/20220901-062004
Epoch 1/10
215/215 [==============================] - 2s 8ms/step - loss: 0.1730 - accuracy: 0.9485 - val_loss: 0.5580 - val_accuracy: 0.7651
Epoch 2/10
215/215 [==============================] - 2s 7ms/step - loss: 0.1232 - accuracy: 0.9587 - val_loss: 0.6345 - val_accuracy: 0.7533
Epoch 3/10
215/215 [==============================] - 2s 7ms/step - loss: 0.1004 - accuracy: 0.9648 - val_loss: 0.7081 - val_accuracy: 0.7415
Epoch 4/10
215/215 [==============================] - 2s 7ms/step - loss: 0.0865 - accuracy: 0.9699 - val_loss: 0.7571 - val_accuracy: 0.7493
Epoch 5/10
215/215 [==============================] - 2s 7ms/step - loss: 0.0772 - accuracy: 0.9730 - val_loss: 0.8133 - val_accuracy: 0.7441
Epoch 6/10
215/215 [==============================] - 2s 7ms/step - loss: 0.0696 - accuracy: 0.9745 - val_loss: 0.8624 - val_accuracy: 0.7415
Epoch 7/10
215/215 [==============================] - 2s 7ms/step - loss: 0.0638 - 

In [173]:
model_1.evaluate(val_data,val_label)

24/24 [==============================] - 0s 4ms/step - loss: 1.0844 - accuracy: 0.7336


[1.0844117403030396, 0.7335957884788513]

In [174]:
tweets=['earthquake hit california minutes ago', 
        'my car on fire now', 
        'I need a blue car']

In [175]:
model_1.predict(tweets)

array([[0.9993068 ],
       [0.8963213 ],
       [0.05997124]], dtype=float32)

## Retrieve the trained word embeddings:

In [ ]:
weights = model_1.get_layer('embedding').get_weights()[0]
weights

array([[ 0.25929037, -0.06279013, -0.06148194, ...,  0.14954968,
         0.10060261,  0.0653062 ],
       [ 0.24403132, -0.03507703, -0.0686942 , ...,  0.12375301,
         0.10604884,  0.02397103],
       [ 0.26330766, -0.00792165,  0.00596493, ...,  0.15860574,
         0.17530532,  0.0099184 ],
       ...,
       [ 0.08522397,  0.05309212,  0.0784494 , ...,  0.00166323,
        -0.00641821, -0.00086374],
       [ 0.01429316,  0.00259889, -0.01726699, ..., -0.02082995,
        -0.01213987, -0.01905471],
       [ 0.04047877, -0.02340493, -0.01548165, ...,  0.00393052,
         0.012569  , -0.03269402]], dtype=float32)

In [ ]:
weights.shape

(10000, 32)

In [ ]:
vocab = text_vectorizer.get_vocabulary()
vocab[:10]

['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is']

In [ ]:
import io
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [ ]:
# try:
#   from google.colab import files
#   files.download('vectors.tsv')
#   files.download('metadata.tsv')
# except Exception:
#   pass

Then, use [Embedding Projector](http://projector.tensorflow.org/) to Visualize the embeddings

## Analyze with LSTM model:

In [ ]:
inputs = tf.keras.layers.Input(shape= (1), dtype = tf.string)

x = text_vectorizer(inputs)

x = embedding(x)

x = tf.keras.layers.LSTM(64,  return_sequences=True)(x)
x = tf.keras.layers.LSTM(32)(x)

x = tf.keras.layers.Dense(16, activation='relu')(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model_2 = tf.keras.Model(inputs,outputs, name='LSTM')

In [ ]:
model_2.summary()

Model: "LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 15, 32)            320000    
                                                                 
 lstm (LSTM)                 (None, 15, 64)            24832     
                                                                 
 lstm_1 (LSTM)               (None, 32)                12416     
                                                                 
 dense_2 (Dense)             (None, 16)                528       
                                                              

In [ ]:
model_2.compile(loss = tf.keras.losses.BinaryCrossentropy(),
                 optimizer = tf.keras.optimizers.Adam(),
                 metrics= ['accuracy'])

In [ ]:
# !rm -rf './tensorboard/LSTM'

In [ ]:
history_2 = model_2.fit(train_data, 
            train_label, 
            validation_data = (val_data,val_label), 
            epochs=10, 
            callbacks=[create_tensorboard_callback('tensorboard','LSTM')])

Saving TensorBoard log files to: tensorboard/LSTM/20220831-192716
Epoch 1/10
215/215 [==============================] - 11s 30ms/step - loss: 0.1997 - accuracy: 0.9218 - val_loss: 0.8313 - val_accuracy: 0.7507
Epoch 2/10
215/215 [==============================] - 6s 26ms/step - loss: 0.0811 - accuracy: 0.9685 - val_loss: 0.8949 - val_accuracy: 0.7388
Epoch 3/10
215/215 [==============================] - 5s 25ms/step - loss: 0.0639 - accuracy: 0.9749 - val_loss: 1.2104 - val_accuracy: 0.7441
Epoch 4/10
215/215 [==============================] - 6s 27ms/step - loss: 0.0608 - accuracy: 0.9737 - val_loss: 1.0866 - val_accuracy: 0.7178
Epoch 5/10
215/215 [==============================] - 6s 26ms/step - loss: 0.0509 - accuracy: 0.9772 - val_loss: 1.2095 - val_accuracy: 0.7362
Epoch 6/10
215/215 [==============================] - 5s 24ms/step - loss: 0.0468 - accuracy: 0.9793 - val_loss: 1.5439 - val_accuracy: 0.7362
Epoch 7/10
215/215 [==============================] - 5s 25ms/step - loss: 

In [ ]:
model_2.evaluate(val_data,val_label)

24/24 [==============================] - 0s 7ms/step - loss: 1.8930 - accuracy: 0.7139


[1.8929849863052368, 0.7139107584953308]

### custom tweets:

In [ ]:
tweets=['earthquake hit california minutes ago', 
        'my car on fire now', 
        'I need a blue car']

In [ ]:
model_2.predict(tweets)

array([[0.9999676 ],
       [0.9162924 ],
       [0.00160667]], dtype=float32)

## Analyze with GRU model:

In [ ]:
inputs = tf.keras.layers.Input(shape= (1), dtype = tf.string)

x = text_vectorizer(inputs)

x = embedding(x)

x = tf.keras.layers.GRU(64,  return_sequences=True)(x)
x = tf.keras.layers.GRU(32)(x)

x = tf.keras.layers.Dense(16, activation='relu')(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model_3 = tf.keras.Model(inputs,outputs, name='GRU')

In [ ]:
model_3.summary()

Model: "GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 15, 32)            320000    
                                                                 
 gru (GRU)                   (None, 15, 64)            18816     
                                                                 
 gru_1 (GRU)                 (None, 32)                9408      
                                                                 
 dense_4 (Dense)             (None, 16)                528       
                                                               

In [ ]:
model_3.compile(loss = tf.keras.losses.BinaryCrossentropy(),
                 optimizer = tf.keras.optimizers.Adam(),
                 metrics= ['accuracy'])

In [ ]:
# !rm -rf './tensorboard/RNN'

In [ ]:
history_3 = model_3.fit(train_data, 
            train_label, 
            validation_data = (val_data,val_label), 
            epochs=10, 
            callbacks=[create_tensorboard_callback('tensorboard','GRU')])

Saving TensorBoard log files to: tensorboard/GRU/20220831-192822
Epoch 1/10
215/215 [==============================] - 11s 29ms/step - loss: 0.1503 - accuracy: 0.9431 - val_loss: 0.9846 - val_accuracy: 0.7087
Epoch 2/10
215/215 [==============================] - 5s 24ms/step - loss: 0.0608 - accuracy: 0.9781 - val_loss: 0.8491 - val_accuracy: 0.7205
Epoch 3/10
215/215 [==============================] - 5s 25ms/step - loss: 0.0526 - accuracy: 0.9785 - val_loss: 1.0873 - val_accuracy: 0.7493
Epoch 4/10
215/215 [==============================] - 5s 24ms/step - loss: 0.0416 - accuracy: 0.9822 - val_loss: 1.8826 - val_accuracy: 0.7152
Epoch 5/10
215/215 [==============================] - 5s 25ms/step - loss: 0.0450 - accuracy: 0.9812 - val_loss: 1.4369 - val_accuracy: 0.7283
Epoch 6/10
215/215 [==============================] - 5s 25ms/step - loss: 0.0381 - accuracy: 0.9813 - val_loss: 1.6457 - val_accuracy: 0.7283
Epoch 7/10
215/215 [==============================] - 5s 25ms/step - loss: 0

In [ ]:
model_3.evaluate(val_data,val_label)

24/24 [==============================] - 0s 6ms/step - loss: 1.9038 - accuracy: 0.7415


[1.903810977935791, 0.7414698004722595]

### custom tweets:

In [ ]:
tweets=['earthquake hit california minutes ago', 
        'my car on fire now', 
        'I need a blue car']

In [ ]:
model_3.predict(tweets)

array([[9.9997842e-01],
       [9.9813539e-01],
       [7.9515576e-04]], dtype=float32)

## Analyzie with Bidirectional LSTM model:

In [ ]:
inputs = tf.keras.layers.Input(shape= (1), dtype = tf.string)

x = text_vectorizer(inputs)

x = embedding(x)

x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True))(x)
x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32))(x)


x = tf.keras.layers.Dense(16, activation='relu')(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model_4 = tf.keras.Model(inputs,outputs, name='Bidirectional-LSTM')

In [ ]:
model_4.summary()

Model: "Bidirectional-LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 15, 32)            320000    
                                                                 
 bidirectional (Bidirectiona  (None, 15, 128)          49664     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 64)               41216     
 nal)                                                            
                                                

In [ ]:
model_4.compile(loss = tf.keras.losses.BinaryCrossentropy(),
                 optimizer = tf.keras.optimizers.Adam(),
                 metrics= ['accuracy'])

In [ ]:
# !rm -rf './tensorboard/RNN'

In [ ]:
history_4 = model_4.fit(train_data, 
            train_label, 
            validation_data = (val_data,val_label), 
            epochs=10, 
            callbacks=[create_tensorboard_callback('tensorboard','Bidirectional-LSTM')])

Saving TensorBoard log files to: tensorboard/Bidirectional-LSTM/20220831-192923
Epoch 1/10
215/215 [==============================] - 18s 45ms/step - loss: 0.1011 - accuracy: 0.9629 - val_loss: 1.1875 - val_accuracy: 0.7454
Epoch 2/10
215/215 [==============================] - 8s 37ms/step - loss: 0.0400 - accuracy: 0.9818 - val_loss: 1.4493 - val_accuracy: 0.7205
Epoch 3/10
215/215 [==============================] - 8s 38ms/step - loss: 0.0398 - accuracy: 0.9803 - val_loss: 1.3763 - val_accuracy: 0.7467
Epoch 4/10
215/215 [==============================] - 8s 38ms/step - loss: 0.0375 - accuracy: 0.9809 - val_loss: 1.6042 - val_accuracy: 0.7441
Epoch 5/10
215/215 [==============================] - 9s 44ms/step - loss: 0.0369 - accuracy: 0.9813 - val_loss: 1.8181 - val_accuracy: 0.7310
Epoch 6/10
215/215 [==============================] - 8s 37ms/step - loss: 0.0357 - accuracy: 0.9822 - val_loss: 1.6126 - val_accuracy: 0.7441
Epoch 7/10
215/215 [==============================] - 8s 37ms

In [ ]:
model_4.evaluate(val_data,val_label)

24/24 [==============================] - 0s 10ms/step - loss: 1.5350 - accuracy: 0.7349


[1.5350419282913208, 0.7349081635475159]

### custom tweets:

In [ ]:
tweets=['earthquake hit california minutes ago', 
        'my car on fire now', 
        'I need a blue car']

In [ ]:
model_4.predict(tweets)

array([[7.7287161e-01],
       [4.7660515e-01],
       [7.0527196e-04]], dtype=float32)

## Analyze with Convolutional 1D model:

In [ ]:
inputs = tf.keras.layers.Input(shape= (1), dtype = tf.string)

x = text_vectorizer(inputs)

x = embedding(x)

# x = tf.transpose(x , perm = [0,2,1])

# print(x.shape)

# x = tf.keras.layers.Flatten()(x)
print(x.shape)

x = tf.keras.layers.Conv1D(filters=64, kernel_size=3,padding= 'valid')(x)
print(x.shape)
x = tf.keras.layers.Conv1D(filters=128, kernel_size=3,padding= 'valid')(x)
print(x.shape)

x = tf.keras.layers.GlobalAveragePooling1D()(x)
x = tf.keras.layers.Dense(16, activation='relu')(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model_5 = tf.keras.Model(inputs,outputs, name='Bidirectional-LSTM')

(None, 15, 32)
(None, 13, 64)
(None, 11, 128)


In [ ]:
model_5.summary()

Model: "Bidirectional-LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 15, 32)            320000    
                                                                 
 conv1d (Conv1D)             (None, 13, 64)            6208      
                                                                 
 conv1d_1 (Conv1D)           (None, 11, 128)           24704     
                                                                 
 global_average_pooling1d_1   (None, 128)              0         
 (GlobalAveragePooling1D)                       

In [ ]:
model_5.compile(loss = tf.keras.losses.BinaryCrossentropy(),
                 optimizer = tf.keras.optimizers.Adam(),
                 metrics= ['accuracy'])

In [ ]:
# !rm -rf './tensorboard/RNN'

In [ ]:
history_5 = model_5.fit(train_data, 
            train_label, 
            validation_data = (val_data,val_label), 
            epochs=10, 
            callbacks=[create_tensorboard_callback('tensorboard','Bidirectional-LSTM')])

Saving TensorBoard log files to: tensorboard/Bidirectional-LSTM/20220831-193057
Epoch 1/10
215/215 [==============================] - 3s 12ms/step - loss: 0.1365 - accuracy: 0.9499 - val_loss: 0.9928 - val_accuracy: 0.7349
Epoch 2/10
215/215 [==============================] - 2s 11ms/step - loss: 0.0646 - accuracy: 0.9740 - val_loss: 1.2160 - val_accuracy: 0.7192
Epoch 3/10
215/215 [==============================] - 2s 11ms/step - loss: 0.0466 - accuracy: 0.9774 - val_loss: 1.4520 - val_accuracy: 0.7362
Epoch 4/10
215/215 [==============================] - 2s 11ms/step - loss: 0.0400 - accuracy: 0.9804 - val_loss: 1.6313 - val_accuracy: 0.7362
Epoch 5/10
215/215 [==============================] - 2s 10ms/step - loss: 0.0388 - accuracy: 0.9812 - val_loss: 1.8688 - val_accuracy: 0.7231
Epoch 6/10
215/215 [==============================] - 2s 11ms/step - loss: 0.0356 - accuracy: 0.9799 - val_loss: 1.9609 - val_accuracy: 0.7244
Epoch 7/10
215/215 [==============================] - 2s 11ms/

In [ ]:
model_5.evaluate(val_data,val_label)

24/24 [==============================] - 0s 3ms/step - loss: 2.6490 - accuracy: 0.7034


[2.6489670276641846, 0.7034120559692383]

### custom tweets:

In [ ]:
tweets=['earthquake hit california minutes ago', 
        'my car on fire now', 
        'I need a blue car']

In [ ]:
model_5.predict(tweets)

array([[0.99974126],
       [0.9929791 ],
       [0.22827303]], dtype=float32)

## Analyze with Universal Sentence Encoder tensorflow hub:

In [ ]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
embed_sample = embed(["The quick brown fox jumps over the lazy dog.",
                      "I am a sentence for which I would like to get its embedding"])

print(embed_sample)

# The following are example embedding output of 512 dimensions per sentence
# Embedding for: The quick brown fox jumps over the lazy dog.
# [-0.03133016 -0.06338634 -0.01607501, ...]
# Embedding for: I am a sentence for which I would like to get its embedding.
# [0.05080863 -0.0165243   0.01573782, ...]


tf.Tensor(
[[-0.03133017 -0.06338634 -0.01607501 ... -0.03242778 -0.0457574
   0.05370456]
 [ 0.05080861 -0.01652434  0.01573779 ...  0.00976657  0.03170121
   0.01788118]], shape=(2, 512), dtype=float32)


In [ ]:
embed(train_data)

<tf.Tensor: shape=(6851, 512), dtype=float32, numpy=
array([[ 0.01178656, -0.00748094,  0.00952065, ...,  0.04002216,
        -0.07293098,  0.01045   ],
       [ 0.08340861, -0.03587737,  0.01278975, ..., -0.00764408,
        -0.06952015, -0.08435119],
       [ 0.03161547,  0.01725737, -0.00808266, ..., -0.03697908,
         0.03299483, -0.04934364],
       ...,
       [-0.03393723, -0.06167451,  0.04978931, ..., -0.05219645,
        -0.05812604, -0.0630734 ],
       [ 0.0215332 ,  0.06060665, -0.07061633, ..., -0.04428503,
        -0.00652964, -0.00850323],
       [ 0.02649508, -0.05326023, -0.05051778, ..., -0.01227468,
         0.01156288, -0.00953933]], dtype=float32)>

### Hub layer is trainable:

In [27]:
encoder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                               input_shape=[],
                               trainable=True,
                               dtype=tf.string,
                               name= 'USE')

In [198]:
#Functional API:

# inputs = tf.keras.layers.Input(shape= [], dtype = tf.string)

# x = encoder_layer(inputs)

# x = tf.keras.layers.Dense(64, activation='relu')(x)
# x = tf.keras.layers.Dense(32, activation='relu')(x)

# outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

# model_6 = tf.keras.Model(inputs,outputs)

In [28]:
model_6 = tf.keras.Sequential([
    encoder_layer,
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')],
    name = 'USE_pretrained_model'
)

In [29]:
model_6.summary()

Model: "USE_pretrained_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense (Dense)               (None, 64)                32832     
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 256,832,769
Trainable params: 256,832,769
Non-trainable params: 0
_________________________________________________________________


In [30]:
model_6.compile(loss = tf.keras.losses.BinaryCrossentropy(),
                 optimizer = tf.keras.optimizers.Adam(),
                 metrics= ['accuracy'])

In [32]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-449b88f3-d623-fd57-568c-4d8d417d17a9)


In [33]:
from tensorflow.keras import mixed_precision

policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

In [34]:
#USE trainable:

model_6.fit(train_data, 
            train_label, 
            validation_data = (val_data,val_label), 
            epochs=10)

Epoch 1/10
215/215 [==============================] - 33s 155ms/step - loss: 0.4246 - accuracy: 0.8154 - val_loss: 0.3991 - val_accuracy: 0.8281
Epoch 2/10
215/215 [==============================] - 33s 153ms/step - loss: 0.2146 - accuracy: 0.9218 - val_loss: 0.5290 - val_accuracy: 0.7900
Epoch 3/10
215/215 [==============================] - 33s 153ms/step - loss: 0.0730 - accuracy: 0.9747 - val_loss: 0.5724 - val_accuracy: 0.7953
Epoch 4/10
215/215 [==============================] - 34s 156ms/step - loss: 0.0390 - accuracy: 0.9848 - val_loss: 0.5870 - val_accuracy: 0.8163
Epoch 5/10
215/215 [==============================] - 34s 159ms/step - loss: 0.0266 - accuracy: 0.9892 - val_loss: 0.6178 - val_accuracy: 0.7913
Epoch 6/10
215/215 [==============================] - 33s 152ms/step - loss: 0.0212 - accuracy: 0.9910 - val_loss: 0.6030 - val_accuracy: 0.8005
Epoch 7/10
215/215 [==============================] - 33s 154ms/step - loss: 0.0162 - accuracy: 0.9928 - val_loss: 0.7527 - val_ac

In [35]:
model_6.evaluate(val_data,val_label)

24/24 [==============================] - 1s 22ms/step - loss: 0.9924 - accuracy: 0.7953


[0.9923694133758545, 0.7952755689620972]

### Hub layer Not trainable:

In [47]:
encoder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                               input_shape=[],
                               trainable=False,
                               dtype=tf.string,
                               name= 'USE')

In [48]:
model_6 = tf.keras.Sequential([
    encoder_layer,
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')],
    name = 'USE_pretrained_model'
)

In [49]:
model_6.summary()

Model: "USE_pretrained_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_6 (Dense)             (None, 64)                32832     
                                                                 
 dense_7 (Dense)             (None, 32)                2080      
                                                                 
 dense_8 (Dense)             (None, 1)                 33        
                                                                 
Total params: 256,832,769
Trainable params: 34,945
Non-trainable params: 256,797,824
_________________________________________________________________


In [50]:
model_6.compile(loss = tf.keras.losses.BinaryCrossentropy(),
                 optimizer = tf.keras.optimizers.Adam(),
                 metrics= ['accuracy'])

In [51]:
# !rm -rf './tensorboard/USE_pretrained_model'
# !rm -rf './checkpoint/'

In [52]:
checkpointPath='./checkpoint/USE'
modelCkechpoint = tf.keras.callbacks.ModelCheckpoint(checkpointPath, 
                                                      monitor='val_accuracy',
                                                      verbose=1,
                                                      save_best_only=True,
                                                      save_weights_only=True)

In [53]:
modelEarlystop = tf.keras.callbacks.EarlyStopping(
                                                monitor='val_accuracy',
                                                min_delta=0,
                                                patience=15,
                                                verbose=1)

In [54]:
history_6 = model_6.fit(train_data, 
            train_label, 
            validation_data = (val_data,val_label), 
            epochs=100, 
            callbacks=[create_tensorboard_callback('tensorboard','USE_pretrained_model'),
                       modelCkechpoint,
                       modelEarlystop])

Saving TensorBoard log files to: tensorboard/USE_pretrained_model/20220901-073300
Epoch 1/100
214/215 [============================>.] - ETA: 0s - loss: 0.4873 - accuracy: 0.7903
Epoch 1: val_accuracy improved from -inf to 0.81627, saving model to ./checkpoint/USE
215/215 [==============================] - 18s 33ms/step - loss: 0.4873 - accuracy: 0.7902 - val_loss: 0.4117 - val_accuracy: 0.8163
Epoch 2/100
214/215 [============================>.] - ETA: 0s - loss: 0.4046 - accuracy: 0.8229
Epoch 2: val_accuracy improved from 0.81627 to 0.82546, saving model to ./checkpoint/USE
215/215 [==============================] - 7s 31ms/step - loss: 0.4045 - accuracy: 0.8229 - val_loss: 0.3981 - val_accuracy: 0.8255
Epoch 3/100
213/215 [============================>.] - ETA: 0s - loss: 0.3878 - accuracy: 0.8344
Epoch 3: val_accuracy did not improve from 0.82546
215/215 [==============================] - 3s 12ms/step - loss: 0.3877 - accuracy: 0.8343 - val_loss: 0.3997 - val_accuracy: 0.8202
Epoc

In [55]:
model_6.evaluate(val_data,val_label)

24/24 [==============================] - 0s 10ms/step - loss: 0.8037 - accuracy: 0.7756


[0.8036916851997375, 0.7755905389785767]

## Loading Model that best classified val_data:

In [58]:
best_model = tf.keras.models.clone_model(model_6)
best_model.load_weights(checkpointPath)

In [60]:
best_model.summary()

Model: "USE_pretrained_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_6 (Dense)             (None, 64)                32832     
                                                                 
 dense_7 (Dense)             (None, 32)                2080      
                                                                 
 dense_8 (Dense)             (None, 1)                 33        
                                                                 
Total params: 256,832,769
Trainable params: 34,945
Non-trainable params: 256,797,824
_________________________________________________________________


In [61]:
best_model.compile(loss = tf.keras.losses.BinaryCrossentropy(),
                 optimizer = tf.keras.optimizers.Adam(),
                 metrics= ['accuracy'])

In [62]:
best_model.evaluate(val_data,val_label)

24/24 [==============================] - 1s 10ms/step - loss: 0.3981 - accuracy: 0.8255


[0.3980754613876343, 0.8254593014717102]

### custom tweets:

In [63]:
tweets=['earthquake hit california minutes ago', 
        'my car on fire now', 
        'I need a blue car']

In [64]:
best_model.predict(tweets)

array([[0.9653 ],
       [0.7324 ],
       [0.09314]], dtype=float16)

## Finding most wrong predicted tweets:

In [77]:
train_data

array(["'McFadden Reportedly to Test Hamstring Thursday' via @TeamStream http://t.co/jWq4KvJH2j",
       'w--=-=-=-[ NEMA warns Nigerians to prepare for drought http://t.co/5uoOPhSqU3',
       "When I was cooking earlier I got electrocuted some crucial ?????? now I'm psychic lol",
       ...,
       'Omron HEM-712C Automatic Blood Pressure Monitor STANDARD AND LARGE BP CUFFS http://t.co/gJBAInQWN9 http://t.co/jPhgpL1c5x',
       'Officials say a quarantine is in place at an Alabama home over a possible Ebola case after developing symptoms... http://t.co/rqKK15uhEY',
       'I moved to England five years ago today. What a whirlwind of time it has been! http://t.co/eaSlGeA1B7'],
      dtype=object)

In [66]:
train_label

array([0, 1, 0, ..., 0, 1, 1])

In [67]:
pred_probs = np.squeeze(best_model.predict(train_data))
print(pred_probs)
pred_label = np.squeeze(np.round(pred_probs))
pred_label

[0.2227 0.8965 0.1377 ... 0.1289 0.9526 0.571 ]


array([0., 1., 0., ..., 0., 1., 1.], dtype=float16)

In [68]:
comparison_table = pd.DataFrame({'tweets':train_data,
              'target': train_label,
              'pred': pred_label,
              'prob': pred_probs})

comparison_table

,tweets,target,pred,prob
0,'McFadden Reportedly to Test Hamstring Thursda...,0,0.0,0.222656
1,w--=-=-=-[ NEMA warns Nigerians to prepare for...,1,1.0,0.896484
2,When I was cooking earlier I got electrocuted ...,0,0.0,0.137695
3,I'm On Fire. http://t.co/WATsmxYTVa,0,0.0,0.309082
4,More than 40 families affected by the fatal ou...,1,1.0,0.904785
...,...,...,...,...
6846,@Eganator2000 There aren't many Obliteration s...,0,0.0,0.067566
6847,just had a panic attack bc I don't have enough...,0,0.0,0.077271
6848,Omron HEM-712C Automatic Blood Pressure Monito...,0,0.0,0.128906
6849,Officials say a quarantine is in place at an A...,1,1.0,0.952637


In [69]:
worst_preds = comparison_table[comparison_table['target'] != comparison_table['pred']].sort_values('prob')
worst_preds

,tweets,target,pred,prob
2189,ÛÏ@LordBrathwaite: Everyone Here: Ahh I hate ...,1,0.0,0.039490
6802,This is set to become a huge one month wonder....,1,0.0,0.047241
4065,destroy the free fandom honestly,1,0.0,0.051086
5277,Oh the usual. Mass murder and world domination...,1,0.0,0.051575
5443,@HaydnExists so glad i saved them all at once ...,1,0.0,0.051941
...,...,...,...,...
3591,@TwopTwips make natural disaster reporting mor...,0,1.0,0.965332
243,@adorableappple No reported flooding po in the...,0,1.0,0.965332
5890,Teen Disaster Preparedness Event in Van Nuys A...,0,1.0,0.971680
6548,DISASTER AVERTED: Police kill gunman with Û÷h...,0,1.0,0.984375


In [70]:
worst_preds[:10]

,tweets,target,pred,prob
2189,ÛÏ@LordBrathwaite: Everyone Here: Ahh I hate ...,1,0.0,0.039490
6802,This is set to become a huge one month wonder....,1,0.0,0.047241
4065,destroy the free fandom honestly,1,0.0,0.051086
5277,Oh the usual. Mass murder and world domination...,1,0.0,0.051575
5443,@HaydnExists so glad i saved them all at once ...,1,0.0,0.051941
333,STAR WARS POWER OF THE JEDI COLLECTION 1 BATTL...,1,0.0,0.052643
4273,All Hail Shadow (Hybrid Mix Feat. Mike Szuter)...,1,0.0,0.053925
523,Do you feel like you are sinking in low self-i...,1,0.0,0.056244
2320,Why are you deluged with low self-image? Take ...,1,0.0,0.056335
5800,@leedsrouge Love what you picked! We're playin...,1,0.0,0.056641


In [71]:
worst_preds[-10:]

,tweets,target,pred,prob
799,Two Jewish Terrorists Charged In Historic-Chur...,0,1.0,0.949707
3041,Mourning notices for stabbing arson victims st...,0,1.0,0.950684
2363,We should all have a fire safety plan. RT @Mat...,0,1.0,0.951660
4438,Trusting Iran to stop terrorism is like inviti...,0,1.0,0.960449
4653,#hot C-130 specially modified to land in a st...,0,1.0,0.963379
3591,@TwopTwips make natural disaster reporting mor...,0,1.0,0.965332
243,@adorableappple No reported flooding po in the...,0,1.0,0.965332
5890,Teen Disaster Preparedness Event in Van Nuys A...,0,1.0,0.971680
6548,DISASTER AVERTED: Police kill gunman with Û÷h...,0,1.0,0.984375
5582,Kosciusko police investigating pedestrian fata...,0,1.0,0.989258


### 5 most wrong disaster tweets:


In [72]:
for i in np.arange(0,5):
  print(worst_preds.iloc[i,0],'\n')
  print(f'target: {worst_preds.iloc[i,1]}, prediction: {worst_preds.iloc[i,2]}, prob: {worst_preds.iloc[i,3]} \n')
  print('========================')

ÛÏ@LordBrathwaite: Everyone Here: Ahh I hate snow!

Me: Lol u call this a snowstorm..?

#growingupincoloradoÛ 

target: 1, prediction: 0.0, prob: 0.03948974609375 

This is set to become a huge one month wonder. (And then Pornhub will be inundated....) http://t.co/gghfx8PzMh 

target: 1, prediction: 0.0, prob: 0.0472412109375 

destroy the free fandom honestly 

target: 1, prediction: 0.0, prob: 0.05108642578125 

Oh the usual. Mass murder and world domination plans over coffee. How's your day going? 

target: 1, prediction: 0.0, prob: 0.05157470703125 

@HaydnExists so glad i saved them all at once then didnÛªt want you stealing my thunder :P 

target: 1, prediction: 0.0, prob: 0.05194091796875 



### 5 most wrong Non-disaster tweets:


In [73]:
for i in np.arange(1,6)*-1:
  print(worst_preds.iloc[i,0],'\n')
  print(f'target: {worst_preds.iloc[i,1]}, prediction: {worst_preds.iloc[i,2]}, prob: {worst_preds.iloc[i,3]} \n')
  print('========================')

Kosciusko police investigating pedestrian fatality hit by a train Thursday http://t.co/m5djLLxoZP 

target: 0, prediction: 1.0, prob: 0.9892578125 

DISASTER AVERTED: Police kill gunman with Û÷hoax deviceÛª atåÊcinema http://t.co/5NG0FzpVdS 

target: 0, prediction: 1.0, prob: 0.984375 

Teen Disaster Preparedness Event in Van Nuys August 11 @ 5:30pm http://t.co/fXUX987vZx via @VanNuysCouncil 

target: 0, prediction: 1.0, prob: 0.9716796875 

@adorableappple No reported flooding po in the area. Ten-4. #mmda 

target: 0, prediction: 1.0, prob: 0.96533203125 

@TwopTwips make natural disaster reporting more interesting by adding 'The' to headlines such as 'Rescuers are sifting through the wreckage' 

target: 0, prediction: 1.0, prob: 0.96533203125 



## Classifying Test Data:
### it has no target

In [75]:
test_df

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan
...,...,...,...,...
3258,10861,NaN,NaN,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...
3259,10865,NaN,NaN,Storm in RI worse than last hurricane. My city...
3260,10868,NaN,NaN,Green Line derailment in Chicago http://t.co/U...
3261,10874,NaN,NaN,MEG issues Hazardous Weather Outlook (HWO) htt...


In [78]:
pred_probs = np.squeeze(best_model.predict(test_df['text']))
pred_label = np.squeeze(np.round(pred_probs))
pred_label

array([1., 1., 1., ..., 1., 1., 1.], dtype=float16)

In [80]:
predicted_notarget_tweets = pd.DataFrame({'tweets':test_df['text'],
              'pred': pred_label,
              'prob': pred_probs})

predicted_notarget_tweets

,tweets,pred,prob
0,Just happened a terrible car crash,1.0,0.739746
1,"Heard about #earthquake is different cities, s...",1.0,0.923340
2,"there is a forest fire at spot pond, geese are...",1.0,0.858887
3,Apocalypse lighting. #Spokane #wildfires,1.0,0.930176
4,Typhoon Soudelor kills 28 in China and Taiwan,1.0,0.968750
...,...,...,...
3258,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...,1.0,0.881836
3259,Storm in RI worse than last hurricane. My city...,1.0,0.968750
3260,Green Line derailment in Chicago http://t.co/U...,1.0,0.904785
3261,MEG issues Hazardous Weather Outlook (HWO) htt...,1.0,0.651855


In [82]:
predicted_notarget_tweets['pred'].value_counts()

0.0    1955
1.0    1308
Name: pred, dtype: int64

In [81]:
predicted_notarget_tweets.sort_values('pred')

,tweets,pred,prob
1631,You were just waiting for us to go down in flames,0.0,0.211792
2696,Man united is not just about playing football ...,0.0,0.161865
1530,I would rather dwell in the land of famine and...,0.0,0.146973
2698,Gets violently ill and it's obvious\nMale cowo...,0.0,0.107849
2699,Renewing Your Kitchen Cabinets for $100 http:/...,0.0,0.097168
...,...,...,...
1600,Truck driver died in Turnpike fire most lanes ...,1.0,0.976074
1598,Pickup Truck Fire On Driscoll Bridge Snags Gar...,1.0,0.870605
1597,Truck catches fire in Warwick: A tree service ...,1.0,0.955078
1529,NONSENSE &gt;&gt; famine memories -- strong e...,1.0,0.916016


In [85]:
predicted_notarget_tweets['pred'] = predicted_notarget_tweets['pred'].map({0.0:'No disaster',1.0:'disaster'})

In [87]:
predicted_notarget_tweets.head(20)

,tweets,pred,prob
0,Just happened a terrible car crash,disaster,0.739746
1,"Heard about #earthquake is different cities, s...",disaster,0.923340
2,"there is a forest fire at spot pond, geese are...",disaster,0.858887
3,Apocalypse lighting. #Spokane #wildfires,disaster,0.930176
4,Typhoon Soudelor kills 28 in China and Taiwan,disaster,0.968750
5,We're shaking...It's an earthquake,disaster,0.646484
6,They'd probably still show more life than Arse...,No disaster,0.083862
7,Hey! How are you?,No disaster,0.075317
8,What a nice hat?,No disaster,0.068054
9,Fuck off!,No disaster,0.085571
